In [2]:
import os
import pandas as pd
import numpy as np
import re
import glob
from Bio import SeqIO

In [3]:
os.chdir('/fs/ess/PAS0439/MING/virome/checkv_res')

In [4]:
checkv = glob.glob('**/quality_summary.tsv', recursive=True)

In [5]:
checkv_list = []
for res in checkv:
    df = pd.read_csv(res, sep = '\t').query('contig_length > 5000')[['contig_id', 'checkv_quality']]
    checkv_list.append(df)
checkv_df = pd.concat(checkv_list)

In [6]:
checkv_df

,contig_id,checkv_quality
0,HS.02_5364||0_partial,Medium-quality
1,HS.02_58486||0_partial,High-quality
2,HS.02_97635||0_partial,Low-quality
3,HS.02_97635||1_partial,Low-quality
4,HS.02_100590||0_partial,Not-determined
...,...,...
962,SRR12444893_1410895||full,Low-quality
963,SRR12444893_1411881||full,Low-quality
964,SRR12444893_1412606||full,Not-determined
965,SRR12444893_1413576||full,Not-determined


In [7]:
votu = []
records = SeqIO.parse('/fs/ess/PAS0439/MING/virome/checkv_trimmed_for_dowmstream/vOTU/vOTU_vs2_vibrant_intersection.fa', 'fasta')
for record in records:
    votu.append(record.name)

In [8]:
tmp = checkv_df[checkv_df.contig_id.isin(votu)]

In [9]:
for_checkv = list(set(votu) - set(tmp.contig_id))
len(for_checkv)

13148

In [10]:
'ERR3201410_654787||full_1' in for_checkv

True

In [139]:
votu_quality = checkv_df[checkv_df.contig_id.isin(votu)].reset_index(drop = True)

,contig_id,checkv_quality


In [151]:
with open('/fs/ess/PAS0439/MING/virome/checkv_trimmed_for_dowmstream/checkv/prophage_for_checkv.fasta', 'w') as outfile:
    records = SeqIO.parse('/fs/ess/PAS0439/MING/virome/checkv_trimmed_for_dowmstream/vOTU/vOTU_vs2_vibrant_intersection.fa', 'fasta')
    for record in records:
        if record.name in for_checkv:
            SeqIO.write(record, outfile, 'fasta')